In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as ipy

In [ ]:

@ipy.interact(i=ipy.widgets.IntSlider())
def f(i):
    img_m = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/proba_map{}.png'.format(i))
    img_b = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/binary_map{}.png'.format(i))
    plt.subplot(1,2,1)
    plt.imshow(img_m)
    plt.subplot(1,2,2)
    plt.imshow(img_b)
    plt.show()